# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse        Shipments  \
0  Nov  1 2022  1:05PM  249802        10        PRF-40775   
1  Nov  1 2022  1:05PM  249802        10        PRF-40778   
2  Nov  1 2022  1:05PM  249802        10        PRF-40783   
3  Nov  1 2022  1:05PM  249802        10        PRF-40786   
4  Nov  1 2022 12:54PM  249801        19  TR-RIH-11022022   
5  Nov  1 2022 12:34PM  249798        12           HH8265   
6  Nov  1 2022 12:20PM  249797        19      ADV80007215   
7  Nov  1 2022 12:20PM  249797        19      ADV80007217   
8  Nov  1 2022 12:20PM  249797        19      ADV80007218   
9  Nov  1 2022 12:20PM  249797        19      ADV80007219   

                 Customer ShipmentStatus  
0                 Bio-PRF       Released  
1                 Bio-PRF       Released  
2                 Bio-PRF       Released  
3                 Bio-PRF       Released  
4  GCH Granules USA, Inc.       Released  
5               Hush Hush       Released  
6     AdvaGen Pharma, Ltd       Released  
7     AdvaGen Pharma, Ltd       Released  
8     AdvaGen Pharma, Ltd      Executing  
9     AdvaGen Pharma, Ltd      Executing

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
11  249797      Executing          3
12  249797       Released          8
13  249798       Released          1
14  249801       Released          1
15  249802       Released          4

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
249795                NaN        NaN       1.0
249797                NaN        3.0       8.0
249798                NaN        NaN       1.0
249801                NaN        NaN       1.0
249802                NaN        NaN       4.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
249750               31.0       27.0       1.0
249757                0.0        0.0       1.0
249760                0.0        0.0       3.0
249774                0.0        0.0       1.0
249783                0.0        0.0       2.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
249750                 31         27         1
249757                  0          0         1
249760                  0          0         3
249774                  0          0         1
249783                  0          0         2

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               249750         31         27         1
1               249757          0          0         1
2               249760          0          0         3
3               249774          0          0         1
4               249783          0          0         2

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing  Released
0               249750        31        27         1
1               249757                             1
2               249760                             3
3               249774                             1
4               249783                             2

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                Customer
0   Nov  1 2022  1:05PM  249802        10                 Bio-PRF
4   Nov  1 2022 12:54PM  249801        19  GCH Granules USA, Inc.
5   Nov  1 2022 12:34PM  249798        12               Hush Hush
6   Nov  1 2022 12:20PM  249797        19     AdvaGen Pharma, Ltd
17  Nov  1 2022 12:05PM  249795        21       NBTY Global, Inc.
18  Nov  1 2022 12:05PM  249794        21       NBTY Global, Inc.
19  Nov  1 2022 12:04PM  249793        21       NBTY Global, Inc.
20  Nov  1 2022 11:30AM  249788        12        SD Head USA, LLC
21  Nov  1 2022 11:13AM  249783        10          Citieffe, Inc.
23  Nov  1 2022  9:47AM  249774        12        SD Head USA, LLC

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                Customer Completed  \
0  Nov  1 2022  1:05PM  249802        10                 Bio-PRF             
1  Nov  1 2022 12:54PM  249801        19  GCH Granules USA, Inc.             
2  Nov  1 2022 12:34PM  249798        12               Hush Hush             
3  Nov  1 2022 12:20PM  249797        19     AdvaGen Pharma, Ltd             
4  Nov  1 2022 12:05PM  249795        21       NBTY Global, Inc.             
5  Nov  1 2022 12:05PM  249794        21       NBTY Global, Inc.             
6  Nov  1 2022 12:04PM  249793        21       NBTY Global, Inc.             
7  Nov  1 2022 11:30AM  249788        12        SD Head USA, LLC             
8  Nov  1 2022 11:13AM  249783        10          Citieffe, Inc.             
9  Nov  1 2022  9:47AM  249774        12        SD Head USA, LLC             

  Executing  Released  
0                   4  
1                   1  
2                   1  
3         3         8  
4                   1  
5                   1  
6                   1  
7                   1  
8                   2  
9                   1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                Customer  Released  \
0  Nov  1 2022  1:05PM  249802        10                 Bio-PRF         4   
1  Nov  1 2022 12:54PM  249801        19  GCH Granules USA, Inc.         1   
2  Nov  1 2022 12:34PM  249798        12               Hush Hush         1   
3  Nov  1 2022 12:20PM  249797        19     AdvaGen Pharma, Ltd         8   
4  Nov  1 2022 12:05PM  249795        21       NBTY Global, Inc.         1   
5  Nov  1 2022 12:05PM  249794        21       NBTY Global, Inc.         1   
6  Nov  1 2022 12:04PM  249793        21       NBTY Global, Inc.         1   
7  Nov  1 2022 11:30AM  249788        12        SD Head USA, LLC         1   
8  Nov  1 2022 11:13AM  249783        10          Citieffe, Inc.         2   
9  Nov  1 2022  9:47AM  249774        12        SD Head USA, LLC         1   

  Executing Completed  
0                      
1                      
2                      
3         3            
4                      
5                      
6                      
7                      
8                      
9

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                Customer  Released  \
0  Nov  1 2022  1:05PM  249802        10                 Bio-PRF         4   
1  Nov  1 2022 12:54PM  249801        19  GCH Granules USA, Inc.         1   
2  Nov  1 2022 12:34PM  249798        12               Hush Hush         1   
3  Nov  1 2022 12:20PM  249797        19     AdvaGen Pharma, Ltd         8   
4  Nov  1 2022 12:05PM  249795        21       NBTY Global, Inc.         1   

  Executing Completed  
0                      
1                      
2                      
3         3            
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                Customer  Released  \
0  Nov  1 2022  1:05PM  249802        10                 Bio-PRF         4   
1  Nov  1 2022 12:54PM  249801        19  GCH Granules USA, Inc.         1   
2  Nov  1 2022 12:34PM  249798        12               Hush Hush         1   
3  Nov  1 2022 12:20PM  249797        19     AdvaGen Pharma, Ltd         8   
4  Nov  1 2022 12:05PM  249795        21       NBTY Global, Inc.         1   

   Executing  Completed  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        3.0        NaN  
4        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                Customer  Released  \
0  Nov  1 2022  1:05PM  249802        10                 Bio-PRF         4   
1  Nov  1 2022 12:54PM  249801        19  GCH Granules USA, Inc.         1   
2  Nov  1 2022 12:34PM  249798        12               Hush Hush         1   
3  Nov  1 2022 12:20PM  249797        19     AdvaGen Pharma, Ltd         8   
4  Nov  1 2022 12:05PM  249795        21       NBTY Global, Inc.         1   

   Executing  Completed  
0        0.0        0.0  
1        0.0        0.0  
2        0.0        0.0  
3        3.0        0.0  
4        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                        
10         749345         9        0.0        0.0
12         749360         3        0.0        0.0
15         249750         1       27.0       31.0
16         249757         1        0.0        0.0
19         499598         9        3.0        0.0
21         749382         3        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse      id  Released  Executing  Completed
0        10  749345         9        0.0        0.0
1        12  749360         3        0.0        0.0
2        15  249750         1       27.0       31.0
3        16  249757         1        0.0        0.0
4        19  499598         9        3.0        0.0
5        21  749382         3        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10         9        0.0        0.0
1        12         3        0.0        0.0
2        15         1       27.0       31.0
3        16         1        0.0        0.0
4        19         9        3.0        0.0
5        21         3        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released    9.0
1        12  Released    3.0
2        15  Released    1.0
3        16  Released    1.0
4        19  Released    9.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse   10   12    15   16   19   21
Status                                  
Completed  0.0  0.0  31.0  0.0  0.0  0.0
Executing  0.0  0.0  27.0  0.0  3.0  0.0
Released   9.0  3.0   1.0  1.0  9.0  3.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status   10   12    15   16   19   21
0          Completed  0.0  0.0  31.0  0.0  0.0  0.0
1          Executing  0.0  0.0  27.0  0.0  3.0  0.0
2           Released  9.0  3.0   1.0  1.0  9.0  3.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status   10   12    15   16   19   21
0  Completed  0.0  0.0  31.0  0.0  0.0  0.0
1  Executing  0.0  0.0  27.0  0.0  3.0  0.0
2   Released  9.0  3.0   1.0  1.0  9.0  3.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()